# Assignment 4

## 1.Using the same Dmart file, calculate the weekly returns for the entire year 

2.Populate a table of returns with week number of month(1,2,3,4,5) and month name        

Please note:
1.Week starts from Monday
2. Use while loop to loop through the dataframe and not any form of groupby, directly or indirectly
3. You buy at the open of the first day of the week and sell at the close of the last day of week
4. If the end of month falls on Wednesday, you exit your position on Wednesday close. New week of the new month starts from Thursday


In [1]:
import pandas as pd
import numpy as np
import calendar
import datetime
from dateutil import parser

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Please change the path of the file as per your pc after r'
path = r'C:\Users\admin\Desktop\Sakshi\6. Python\Assignment 4\Dmart.csv'

##### Dmart file = df

In [3]:
df = pd.read_csv(path)

In [4]:
#converted date/time column to date/time format

df['Date'] = np.nan

for i in range (0,len(df['Date/Time'])):
    date_string = df['Date/Time'][i]
    date_object = parser.parse(date_string)
    df['Date'][i] = date_object

In [5]:
#Adding a day column, month column, week number column
df['Day'] = np.nan
df['Month'] = np.nan
df['Week Number'] = np.nan

 
for i in range (0,len(df['Date/Time'])):
    df['Day'][i] = (df['Date'][i]).strftime("%A")
    df['Month'][i] = (df['Date'][i]).strftime("%m")
    df['Week Number'][i] = (df['Date'][i]).strftime("%W")  


In [6]:
#adding a column to check if day is wednesday or no

df['Checking day is wednesday or no'] = df['Day'].apply (lambda x: '1' if x == 'Wednesday' else '0')

In [7]:
#adjusting month to another cell
i = 0
while i < len(df['Date/Time']):
    for i in range (0,len(df['Date/Time'])):
        if i == 251:
            df.loc[i,'NewLocation'] = df.loc[i,'Month']
        else:
            df.loc[i,'NewLocation'] = df.loc[i+1,'Month']
        i += 1

In [8]:
#finding month end
i = 0
while i < len(df['Date/Time']):
    for i in range (0,len(df['Date/Time'])):
        if df.loc[i,'Month'] == df.loc[i,'NewLocation']:
            df.loc[i,'MonthCheck'] = '0'
        else:
            df.loc[i,'MonthCheck'] = '1'
        i += 1

In [9]:
# combining month end and wednesday close
i = 0
while i < len(df['Date/Time']):
    for i in range (0,len(df['Date/Time'])):
        if ((df.loc[i,'MonthCheck'] == '1') and (df.loc[i,'Checking day is wednesday or no'] == '1')):
            df.loc[i,'WednesdayClose'] = '1'
        else:
            df.loc[i,'WednesdayClose'] = '0'
        i += 1    

In [10]:
#close counter
i = 0
while i < len(df['Date/Time']):
    for i in range (0,len(df['Date/Time'])):
        if i == 0:
            df.loc[i,'CloseApprox'] = '0'
        elif i == 251:
            df.loc[i,'CloseApprox'] = '1'
        elif (df.loc[i,'Week Number']) == (df.loc[i+1,'Week Number']):
            df.loc[i,'CloseApprox'] = '0'
        else:
            df.loc[i,'CloseApprox'] = '1'
        i += 1    

In [11]:
#to determine the close
i = 0
while i < len(df['Date/Time']):
    for i in range (0,len(df['Date/Time'])):
        if df.loc[i,'WednesdayClose'] == '1':
            df.loc[i,'CloseCount'] = 1
        elif df.loc[i,'CloseApprox'] == '1':
            df.loc[i,'CloseCount'] = 1
        else:
            df.loc[i,'CloseCount'] = 0
        i += 1

In [12]:
#to determine the open count
i = 0
while i < len(df['Date/Time']):
    for i in range (0,len(df['Date/Time'])):
        if i == 0:
            df.loc[i,'OpenCount'] = 1
        elif i == 251:
            df.loc[i,'OpenCount'] = 0
        elif df.loc[i,'Week Number'] != df.loc[i-1,'Week Number']:
            df.loc[i,'OpenCount'] = 1
        else:
            df.loc[i,'OpenCount'] = 0
        i += 1

In [13]:
#counters of close and open
i = 0
summationclose = 0
for i in range(0,len(df['Date/Time'])):
    summationclose += df.loc[i,'CloseCount']
    df.loc[i,'CloseCounter'] = summationclose
    
j = 0
summationopen = 0
for j in range (0,len(df['Date/Time'])):
    summationopen += df.loc[j,'OpenCount']
    df.loc[j,'OpenCounter'] = summationopen

In [14]:
#close counter helper
i = 0
while i < len(df):
    for i in range(0,len(df['Date/Time'])):
        if i == 0:
            df.loc[i,'CloseCounterHelper'] = df.loc[i,'CloseCounter'] 
        elif df.loc[i,'CloseCounter'] != df.loc[i,'OpenCounter']:
            df.loc[i,'CloseCounterHelper'] = df.loc[i-1,'CloseCounter']
        else:
            df.loc[i,'CloseCounterHelper'] = df.loc[i,'CloseCounter']
        i+=1

In [15]:
#creating the final open count
i = 0
while i < len(df):
    for i in range (0,len(df['Date/Time'])):
        if i == 0:
            df.loc[i,'OpenCounterFinal'] = df.loc[i,'OpenCounter']
        elif df.loc[i,'OpenCounter'] != df.loc[i-1,'OpenCounter']:
            df.loc[i,'OpenCounterFinal'] = df.loc[i,'OpenCounter']
        else:
            df.loc[i,'OpenCounterFinal'] = 0
        i+= 1

In [16]:
#reaching a step closer to close count final
i = 0
while i < len(df):
    for i in range (0,len(df['Date/Time'])):
        if i == 0:
            df.loc[i,'CloseCounterFinal'] = df.loc[i,'CloseCounterHelper']
        elif i == 251:
            df.loc[i,'CloseCounterFinal'] = df.loc[i,'CloseCounterHelper']
        elif (df.loc[i,'CloseCounterHelper'] == df.loc[i,'OpenCounterFinal']):
            df.loc[i,'CloseCounterFinal'] = 0
        elif (df.loc[i,'CloseCounterHelper'] != df.loc[i-1,'CloseCounterHelper']):
            df.loc[i,'CloseCounterFinal'] = df.loc[i,'CloseCounterHelper']
        elif (df.loc[i,'CloseCounterHelper'] != df.loc[i+1,'CloseCounterHelper']):
            df.loc[i,'CloseCounterFinal'] = df.loc[i,'CloseCounterHelper']
        else:
            df.loc[i,'CloseCounterFinal'] = 0
        i += 1

In [17]:
#the final open count
i = 0
while i < len(df):
    for i in range (0,len(df['Date/Time'])):
        if i == 251:
            df.loc[i,'CloseFinal1'] = df.loc[i,'CloseCounterFinal']
        elif df.loc[i+1,'CloseCounterFinal'] != 0:
            df.loc[i,'CloseFinal1'] = 0
        else:
            df.loc[i,'CloseFinal1'] = df.loc[i,'CloseCounterFinal']
        i += 1

### thus the main open counter column = 'OpenCounterFinal' and for close it is 'CloseFinal1' 

In [18]:
i = 0
while i < len(df):
    for i in range (0,len(df['Date/Time'])):
        if df.loc[i,'OpenCounterFinal'] != 0:
            df.loc[i,'BuyPrice'] = df.loc[i,'Open']
        else:
            df.loc[i,'BuyPrice'] = 0
        i += 1

j = 0
while j < len(df):
    for j in range (0,len(df['Date/Time'])):
        if df.loc[j,'CloseFinal1'] != 0:
            df.loc[j,'SellPrice'] = df.loc[j,'Close']
        else:
            df.loc[j,'SellPrice'] = 0
        j += 1
        
#display(df)

In [19]:
BuyData = pd.DataFrame({
    'Buy':[],
    'Month': []})

i = 0
while i < len(df):
    for i in range (0,len(df)):
        if df.loc[i,'BuyPrice'] != 0:
            BuyData.loc[i,'Buy'] = df.loc[i,'BuyPrice']
            BuyData.loc[i,'Month'] = df.loc[i,'Month']
            
        i+=1
        
BuyData.reset_index(inplace = True,drop = True)           
#display(BuyData)


In [20]:
SellData = pd.DataFrame({
    'Sell':[]})

i = 0
while i < len(df):
    for i in range (0,len(df)):
        if df.loc[i,'SellPrice'] != 0:
            SellData.loc[i,'Sell'] = df.loc[i,'SellPrice']
        i+=1
        
SellData.reset_index(inplace = True,drop = True)            
#display(SellData)

In [21]:
Returns = pd.merge(BuyData,SellData,left_index = True,right_index = True)

#display(Returns)

In [22]:
Returns['ROC'] = ((Returns['Sell']/Returns['Buy'])-1)*100
#display(Returns)

In [40]:
Table1 = pd.DataFrame({'January':[]})

i = 0
while i < len(Returns):
    for i in range (0,len(Returns)):
        if Returns.loc[i,'Month'] == '01':
            Table1.loc[i,'January'] = Returns.loc[i,'ROC']
        i+=1   
display(Table1)

,January
0,-0.332486
1,2.733990
2,3.405195
3,-2.570000
4,6.504755


In [24]:
Table2 = pd.DataFrame({'February':[]})

i = 0
while i < len(Returns):
    for i in range (0,len(Returns)):
        if Returns.loc[i,'Month'] == '02':
            Table2.loc[i,'February'] = Returns.loc[i,'ROC']
        i+=1   
display(Table2)

,February
5,9.750480
6,5.296870
7,4.593332
8,-5.098416


In [25]:
Table3 = pd.DataFrame({'March':[]})

i = 0
while i < len(Returns):
    for i in range (0,len(Returns)):
        if Returns.loc[i,'Month'] == '03':
            Table3.loc[i,'March'] = Returns.loc[i,'ROC']
        i+=1   
display(Table3)

,March
9,-3.976445
10,-3.844394
11,-4.000601
12,14.250398
13,-0.613069


In [26]:
Table4 = pd.DataFrame({'April':[]})

i = 0
while i < len(Returns):
    for i in range (0,len(Returns)):
        if Returns.loc[i,'Month'] == '04':
            Table4.loc[i,'April'] = Returns.loc[i,'ROC']
        i+=1   
display(Table4)

,April
14,14.475384
15,-4.650862
16,3.612189
17,3.636957


In [27]:
Table5 = pd.DataFrame({'May':[]})

i = 0
while i < len(Returns):
    for i in range (0,len(Returns)):
        if Returns.loc[i,'Month'] == '05':
            Table5.loc[i,'May'] = Returns.loc[i,'ROC']
        i+=1   
display(Table5)

,May
18,-2.389404
19,2.359649
20,2.798971
21,-2.977768


In [28]:
Table6 = pd.DataFrame({'June':[]})

i = 0
while i < len(Returns):
    for i in range (0,len(Returns)):
        if Returns.loc[i,'Month'] == '06':
            Table6.loc[i,'June'] = Returns.loc[i,'ROC']
        i+=1   
display(Table6)

,June
22,7.710875
23,-4.849206
24,-0.415086
25,-3.026205
26,-0.185234


In [29]:
Table7 = pd.DataFrame({'July':[]})

i = 0
while i < len(Returns):
    for i in range (0,len(Returns)):
        if Returns.loc[i,'Month'] == '07':
            Table7.loc[i,'July'] = Returns.loc[i,'ROC']
        i+=1   
display(Table7)

,July
27,1.162892
28,-9.968100
29,2.601141
30,0.951566


In [30]:
Table8 = pd.DataFrame({'August':[]})

i = 0
while i < len(Returns):
    for i in range (0,len(Returns)):
        if Returns.loc[i,'Month'] == '08':
            Table8.loc[i,'August'] = Returns.loc[i,'ROC']
        i+=1   
display(Table8)

,August
31,3.369266
32,1.851852
33,3.596974
34,2.178462
35,-2.064031


In [31]:
Table9 = pd.DataFrame({'September':[]})

i = 0
while i < len(Returns):
    for i in range (0,len(Returns)):
        if Returns.loc[i,'Month'] == '09':
            Table9.loc[i,'September'] = Returns.loc[i,'ROC']
        i+=1   
display(Table9)

,September
36,-1.350220
37,-4.486667
38,-3.324138
39,3.500876


In [32]:
Table10 = pd.DataFrame({'October':[]})

i = 0
while i < len(Returns):
    for i in range (0,len(Returns)):
        if Returns.loc[i,'Month'] == '10':
            Table10.loc[i,'October'] = Returns.loc[i,'ROC']
        i+=1   
display(Table10)

,October
40,-6.172051
41,-4.237452
42,7.490922
43,5.655660


In [33]:
Table11 = pd.DataFrame({'November':[]})

i = 0
while i < len(Returns):
    for i in range (0,len(Returns)):
        if Returns.loc[i,'Month'] == '11':
            Table11.loc[i,'November'] = Returns.loc[i,'ROC']
        i+=1   
display(Table11)

,November
44,2.437500
45,2.953102
46,0.355917
47,-3.045191


In [34]:
Table12 = pd.DataFrame({'December':[]})

i = 0
while i < len(Returns):
    for i in range (0,len(Returns)):
        if Returns.loc[i,'Month'] == '12':
            Table12.loc[i,'December'] = Returns.loc[i,'ROC']
        i+=1   
display(Table12)

,December
48,4.802575
49,9.959458
50,-2.871087
51,1.062382
52,2.890970


In [44]:
#Merging table 1 and 2
#Returns = pd.merge(BuyData,SellData,left_index = True,right_index = True)
H1 = pd.merge(Table1,Table2,left_index=True,right_index=True)
display(H1)

,January,February


In [38]:
#display(df)

In [37]:
#df.to_csv('DmartOutput.csv')
#Data.to_csv('ReturnsOutput.csv')